# New Def

지난번 만든 함수에는 user_list를 만들어서 data가 일부 소실되는 바람에 submission 값도 소실되는 결과가 있었음.

따라서 이번에는 ID 중복을 drop하지 않는 방향으로 진행해 볼 것임

또한, occyp_type을 미리 처리해주고 ID에 occyp_type을 추가하는 시도를 해볼 것임.

+ 함수를 처리하는 중 print로 data를 확인하는 부분이 있는데, 이부분을 color 변경을 통해서 주목성을 높임
  

## 필요한 라이브러리 호출

In [24]:

import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [64]:
# 경로 설정 (dataset을 저장하는 경로임.)
path = 'data/'

train = pd.read_csv('{}train.csv'.format(path))
test = pd.read_csv('{}test.csv'.format(path))

In [65]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    # df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)
    
del df

### 1. 나이, 고용일자를 이용하여 파생변수 형성

In [66]:
# 1st

def year_month_total(df_1, df_2, column, column_name):
    """[나이, 고용연수 파생변수를 생성하기 위한 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        column ([생성할 컬럼의 기준 컬럼 명]): [생성할 컬럼의 기준 컬럼 명을 입력해주세요]
        column_name ([생성할 컬럼의 이름]): [연단위, 연+월단위로 표시된 파생변수 이름입니다.]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    # 입력한 컬럼이 DAYS_BIRTH인 경우 If로 처리
    
    if (column == 'DAYS_BIRTH') == True:    
        for dataset,name in zip(df_list,names):
            dataset['DAYS_BIRTH_MOD'] = dataset[column]-dataset['begin_month']*30
            dataset['{}_year'.format(column_name)] = dataset['DAYS_BIRTH_MOD'].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset['DAYS_BIRTH_MOD'].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            # dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
            dataset['{}_month'.format(column_name)] = dataset['DAYS_BIRTH_MOD'].abs()//30
        
            print('{0}set에 {1}_year, {1}_month, {1}_total, {2} 컬럼을 생성하였습니다. \n'.format(name,column_name,'DAYS_BIRTH_MOD'))
    
    # 입력한 컬럼이 DAYS_EMPLOYED인 경우 else로 처리 (occyp_type 처리를 위한 작업이므로 1번째로 수행)
    
    else:
        for dataset,name in zip(df_list,names):
            dataset['DAYS_EMPLOYED_MOD'] = dataset['DAYS_EMPLOYED']-dataset['begin_month']*30
            print('{}set에 근무일자가 잘못기입된 '.format(name) +
                  '\033[96m' + '{} cases'.format(len(dataset[dataset['DAYS_EMPLOYED_MOD']>=0])) + 
                  '\033[0m' + '에 대한 처리를 시작합니다.')
            # Begin_month 처리한 사람들 대상 (카드를 생성한 시점에 무직일 확률이 높은 사람들 + 애초에 무직)
            dataset.loc[(dataset['DAYS_EMPLOYED_MOD']>0), 'DAYS_EMPLOYED_MOD'] = 0
            a = len(dataset.loc[(dataset['DAYS_EMPLOYED_MOD']==0)])
            # Emplyed 시간이 애초에 없는 사람 대상 (애초에 무직)
            dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].replace(365243, 0)
            b = len(dataset.loc[(dataset['DAYS_EMPLOYED']==0)])
            print('\033[92m'+'원래 무직인 사람들의 수 : {}'.format(b))
            print('카드를 생성한 시점에 무직일 확률일 높은 사람들 : {}'.format(a-b) + '\033[0m')
            dataset['{}_year'.format(column_name)] = dataset['DAYS_EMPLOYED_MOD'].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset['DAYS_EMPLOYED_MOD'].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            # dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
            dataset['{}_month'.format(column_name)] = dataset['DAYS_EMPLOYED_MOD'].abs()//30
        
            print('{0}set에 {1}_year, {1}_month, {1}_total, {2} 컬럼을 생성하였습니다. \n'.format(name,column_name,'DAYS_BIRTH_MOD'))

        
    del df_list,names
    return dataset.head(1)

    

In [67]:
year_month_total(train, test,'DAYS_BIRTH','age')

trainset에 age_year, age_month, age_total, DAYS_BIRTH_MOD 컬럼을 생성하였습니다. 

testset에 age_year, age_month, age_total, DAYS_BIRTH_MOD 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,work_phone,phone,email,occyp_type,family_size,begin_month,DAYS_BIRTH_MOD,age_year,age_month,age_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,1,0,NaN,2.0,-60.0,-20190.0,55.0,673.0,55.25


In [68]:
year_month_total(train, test,'DAYS_EMPLOYED','work')

trainset에 근무일자가 잘못기입된 8357 cases에 대한 처리를 시작합니다.
원래 무직인 사람들의 수 : 4438
카드를 생성한 시점에 무직일 확률일 높은 사람들 : 3919
trainset에 work_year, work_month, work_total, DAYS_BIRTH_MOD 컬럼을 생성하였습니다. 

testset에 근무일자가 잘못기입된 3169 cases에 대한 처리를 시작합니다.
원래 무직인 사람들의 수 : 1697
카드를 생성한 시점에 무직일 확률일 높은 사람들 : 1472
testset에 work_year, work_month, work_total, DAYS_BIRTH_MOD 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,family_size,begin_month,DAYS_BIRTH_MOD,age_year,age_month,age_total,DAYS_EMPLOYED_MOD,work_year,work_month,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,2.0,-60.0,-20190.0,55.0,673.0,55.25,0.0,0.0,0.0,0.0


### 2. 직업 Nan 값 처리

In [69]:
# 2nd

def occyp_type_nan(df_1, df_2):
    """[직업 컬럼 내의 Nan 값을 처리합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # 무직자 처리 함수
    
    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        
        print('직업 컬럼의 ' +
              '\033[92m' + 'Nan value {}명 중 '.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])) +
              '\033[96m' + '{}명이 실제 고용일수가 0일입니다. '.format(len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])) + 
              '\033[0m'+ '따라서 retired로 처리합니다.')
        
        dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'retired'
        
        # 무직자는 아니지만 데이터가 없는 경우
        
        print('나머지 {}명 데이터는 no_data로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])))
        
        dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no_data'
        
        print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names
    return dataset.head(1)

occyp_type_nan(train, test)

직업 컬럼의 Nan value 8171명 중 4438명이 실제 고용일수가 0일입니다. 따라서 retired로 처리합니다.
나머지 3733명 데이터는 no_data로 처리합니다.
trainset의 직업 Nan value를 처리하였습니다.

직업 컬럼의 Nan value 3152명 중 1697명이 실제 고용일수가 0일입니다. 따라서 retired로 처리합니다.
나머지 1455명 데이터는 no_data로 처리합니다.
testset의 직업 Nan value를 처리하였습니다.



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,family_size,begin_month,DAYS_BIRTH_MOD,age_year,age_month,age_total,DAYS_EMPLOYED_MOD,work_year,work_month,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,2.0,-60.0,-20190.0,55.0,673.0,55.25,0.0,0.0,0.0,0.0


### 3. ID컬럼 생성

In [70]:
def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [train 데이터셋을 입력해주세요]
        df_2 ([Dataset]): [test 데이터셋을 입력해주세요]
    """
    
    # 변수 설정
     
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    a = []
    change_credit = 0
    names = ['train', 'test']

    # ID 생성
    
    for name, dataset, in zip(names, df_list):
    # for dataset in df_list:
        dataset['ID'] = dataset['gender'].astype(str) +'.'+ dataset['car'].astype(str)+'.' + dataset['reality'].astype(str)+'.' + \
                dataset['child_num'].astype(str)+'.' + dataset['income_total'].astype(str) +'.'+ dataset['income_type'].astype(str) +'.'+ \
                dataset['edu_type'].astype(str)+'.' + dataset['family_type'].astype(str)+'.' + dataset['house_type'].astype(str) +'.'+ \
                dataset['DAYS_BIRTH'].astype(str) +'.'+ dataset['DAYS_EMPLOYED'].astype(str) +'.'+ dataset['work_phone'].astype(str) +'.'+ \
                dataset['phone'].astype(str) +'.'+ dataset['email'].astype(str) +'.' + dataset['family_size'].astype(str)+'.' + dataset['occyp_type'].astype(str)
        
        dataset = dataset.sort_values(['ID','begin_month'], ascending=[True, False])
        dataset.reset_index(inplace=True)
        dataset.drop(columns={'index'}, inplace=True)

        # OrdinalEncoder로 ID 변환
        
        print('\033[96m' + '해당 Data 내의 고유 ID 수는 {}입니다. '.format(dataset['ID'].nunique()) + 
              '\033[0m' + 'OrdinalEncoder를 이용힌 변환을 시작합니다.')
        encode = OrdinalEncoder()
        dataset[['ID']] = encode.fit_transform(dataset[['ID']])

        # Nth_card 컬럼 추가 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 몇 번째 카드를 만들었는지 생성해준다)
        
        print('Column(Nth_card)을 생성합니다.')
        for i in tqdm(range(dataset['ID'].nunique())):
            Nth_card = len(dataset[dataset['ID']==i]) + 1
            
            while Nth_card > 0:
                Nth_card = Nth_card -1
                a.append(Nth_card)
                if Nth_card == 1:
                    break
                
        dataset['Nth_card'] = a        
        a.clear()
        print('\033[92m' + '신규로 카드를 개설한 사람의 수 : {}'.format(dataset[dataset['begin_month']==0.0]['ID'].nunique()) +
              '\033[0m')
        
        # Credit이 좋아지지 않고 악화된 경우를 체크하는 부분 (begin_month기준으로 Nth_card를 추가해줬으므로 인덱스 값을 기준으로 시행한다.)
        
        if (dataset['ID'].nunique() > 8000)==True:
            print('credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.')
            for i in tqdm(range(dataset['ID'].nunique())):
                num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
                for j in range(len(num)-1):
                    if (num[j] <= num[j+1]) == True:
                        pass
                    else :
                        change_credit += 1
                        break
            print('\033[96m'+'{}명 (전체 인원 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                        (change_credit/dataset['ID'].nunique())*100) + 
                  '\033[0m')  
        else:
            print('\033[96m' + 'credit 정보가 없는 testset입니다.' + 
                  '\033[0m')
        
        # dataset 저장부분
        
        print('dataset(ID_list_{})을 저장합니다. \n'.format(name))    
        dataset.to_csv('{}ID_list_{}.csv'.format(path, name))
        
    
    del names, a
    return dataset.head(5)

In [71]:
ID_col(train, test)

해당 Data 내의 고유 ID 수는 8759입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 8759/8759 [00:04<00:00, 1913.81it/s]


신규로 카드를 개설한 사람의 수 : 214
credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.


100%|██████████| 8759/8759 [00:05<00:00, 1724.20it/s]


1842명 (전체 인원 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.
dataset(ID_list_train)을 저장합니다. 

해당 Data 내의 고유 ID 수는 5585입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 5585/5585 [00:02<00:00, 1980.46it/s]


신규로 카드를 개설한 사람의 수 : 82
credit 정보가 없는 testset입니다.
dataset(ID_list_test)을 저장합니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,DAYS_BIRTH_MOD,age_year,age_month,age_total,DAYS_EMPLOYED_MOD,work_year,work_month,work_total,ID,Nth_card
0,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,-21611.0,59.0,720.0,59.17,0.0,0.0,0.0,0.0,0.0,3
1,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,-20321.0,55.0,677.0,55.67,0.0,0.0,0.0,0.0,0.0,2
2,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,-20111.0,55.0,670.0,55.08,0.0,0.0,0.0,0.0,0.0,1
3,F,N,N,0,103500.0,Pensioner,Higher education,Single / not married,House / apartment,-24543,...,-23703.0,64.0,790.0,64.92,0.0,0.0,0.0,0.0,1.0,1
4,F,N,N,0,103500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-20123,...,-19793.0,54.0,659.0,54.17,0.0,0.0,0.0,0.0,2.0,2


In [72]:
train = pd.read_csv('{}ID_list_train.csv'.format(path))
test = pd.read_csv('{}ID_list_test.csv'.format(path))

### faimly_size 조정

In [73]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # family_size Outlier를 처리하는 작업
    
    for dataset,name in zip(df_list,names):
        
        print('family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.')
        
        dataset.drop(columns=['child_num'], inplace=True)
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                                      len(dataset.loc[dataset['family_size'] >= num_of_outlier])) +
              '\033[0m')
        
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        
        for i in tqdm(range(len(sub))):
        
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        # 잘 처리되었는지 확인하는 부분
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier]))+
              '\033[0m')

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        
        print('1인당 소득으로 소득 수준을 조정합니다.')
        
        dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        
        print('{} 데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
    return dataset.head(1)

replace_value_family(train, test, 7)

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 15


100%|██████████| 15/15 [00:06<00:00,  2.44it/s]


family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train 데이터 셋 처리 완료.

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 10


100%|██████████| 10/10 [00:00<00:00, 10.14it/s]

family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test 데이터 셋 처리 완료.



,Unnamed: 0,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,age_year,age_month,age_total,DAYS_EMPLOYED_MOD,work_year,work_month,work_total,ID,Nth_card,income_mean
0,0,F,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,59.0,720.0,59.17,0.0,0.0,0.0,0.0,0.0,3,51750.0


In [74]:
def contact_num(df_1, df_2):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    for dataset,name in zip(df_list,names):
        dataset['contact_num'] = dataset['FLAG_MOBIL'] + dataset['work_phone'] + dataset['phone'] + dataset['email']
        print('\033[96m' + "{}set에 contact_num 컬럼을 추가하였습니다.".format(name))
        dataset.drop(columns=['FLAG_MOBIL', 'work_phone', 'phone', 'email'], inplace=True)
        print('\033[0m'+"'FLAG_MOBIL', 'work_phone', 'phone', 'email' 컬럼을 삭제하였습니다.")

In [75]:
contact_num(train, test)

trainset에 contact_num 컬럼을 추가하였습니다.
'FLAG_MOBIL', 'work_phone', 'phone', 'email' 컬럼을 삭제하였습니다.
testset에 contact_num 컬럼을 추가하였습니다.
'FLAG_MOBIL', 'work_phone', 'phone', 'email' 컬럼을 삭제하였습니다.


In [76]:
names = ['train', 'test']

for name,dataset in zip(names, [train, test]):
    dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'], inplace=True)
    dataset.to_csv('{}ID_list_{}_ver2.csv'.format(path,name))

## 인코딩 시행

In [51]:
train = pd.read_csv('{}ID_list_train_ver2.csv'.format(path))
test = pd.read_csv('{}ID_list_test_ver2.csv'.format(path))

In [52]:
for df in (train, test):
    df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1',
                     'age_year', 'work_year', 'age_month', 'work_month',
                     'DAYS_BIRTH_MOD','DAYS_EMPLOYED_MOD'], inplace=True)

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gender        26457 non-null  object 
 1   car           26457 non-null  object 
 2   reality       26457 non-null  object 
 3   income_total  26457 non-null  float64
 4   income_type   26457 non-null  object 
 5   edu_type      26457 non-null  object 
 6   family_type   26457 non-null  object 
 7   house_type    26457 non-null  object 
 8   occyp_type    26457 non-null  object 
 9   family_size   26457 non-null  float64
 10  begin_month   26457 non-null  float64
 11  credit        26457 non-null  float64
 12  age_total     26457 non-null  float64
 13  work_total    26457 non-null  float64
 14  ID            26457 non-null  float64
 15  Nth_card      26457 non-null  int64  
 16  income_mean   26457 non-null  float64
 17  contact_num   26457 non-null  int64  
dtypes: float64(8), int64(2), o

In [29]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  9
Number of Categorical features:  8


In [30]:
numerical_feats

['income_total',
 'family_size',
 'begin_month',
 'age_total',
 'work_total',
 'ID',
 'Nth_card',
 'income_mean',
 'contact_num']

In [31]:
categorical_feats

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [32]:
for df in [train,test]:
    df['income_total'] = np.log1p(1+df['income_total'])
    df['income_mean'] = np.log1p(1+df['income_mean'])

In [33]:
encoder = OrdinalEncoder()
train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
test[categorical_feats] = encoder.transform(test[categorical_feats])

In [34]:
from sklearn.cluster import KMeans

kmeans_train = train.drop(['credit'], axis=1)
kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
train['cluster'] = kmeans.predict(kmeans_train)
test['cluster'] = kmeans.predict(test)

In [35]:
numerical_feats.remove('income_total')
scaler = StandardScaler()
train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
test[numerical_feats] = scaler.transform(test[numerical_feats])

In [36]:
train.drop(columns=['ID','occyp_type'], inplace=True)
test.drop(columns=['ID','occyp_type'], inplace=True)

In [41]:
a = ['income_type', 'edu_type', 'family_type', 'house_type', 'gender','car','reality']
train[a] = train[a].astype(int)
test[a] = test[a].astype(int)


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gender        26457 non-null  int64  
 1   car           26457 non-null  int64  
 2   reality       26457 non-null  int64  
 3   income_total  26457 non-null  float64
 4   income_type   26457 non-null  int64  
 5   edu_type      26457 non-null  int64  
 6   family_type   26457 non-null  int64  
 7   house_type    26457 non-null  int64  
 8   family_size   26457 non-null  float64
 9   begin_month   26457 non-null  float64
 10  credit        26457 non-null  float64
 11  age_total     26457 non-null  float64
 12  work_total    26457 non-null  float64
 13  Nth_card      26457 non-null  float64
 14  income_mean   26457 non-null  float64
 15  contact_num   26457 non-null  float64
 16  cluster       26457 non-null  int32  
dtypes: float64(9), int32(1), int64(7)
memory usage: 3.3 MB


In [46]:
n_est = 2000
seed = 42
n_fold = 15
n_class = 3

target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = test

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool

In [47]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type'] #'gender', 'car', 'reality']#, 'occyp_type']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')


----------------- Fold 0 -----------------

Learning rate set to 0.115128


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 1.0354427	test: 1.0361760	best: 1.0361760 (0)	total: 136ms	remaining: 2m 16s
100:	learn: 0.7818156	test: 0.8067317	best: 0.8066847 (99)	total: 6.85s	remaining: 1m
200:	learn: 0.7520353	test: 0.7991288	best: 0.7991288 (200)	total: 13.1s	remaining: 51.9s
300:	learn: 0.7279632	test: 0.7922224	best: 0.7920414 (299)	total: 19.7s	remaining: 45.8s
400:	learn: 0.7073004	test: 0.7873350	best: 0.7873118 (393)	total: 26s	remaining: 38.9s
500:	learn: 0.6878418	test: 0.7842492	best: 0.7840985 (498)	total: 32.2s	remaining: 32.1s
600:	learn: 0.6686790	test: 0.7813809	best: 0.7813367 (598)	total: 38.6s	remaining: 25.6s
700:	learn: 0.6523709	test: 0.7792060	best: 0.7791462 (699)	total: 45s	remaining: 19.2s
800:	learn: 0.6352404	test: 0.7777562	best: 0.7772311 (765)	total: 51.4s	remaining: 12.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7772310533
bestIteration = 765

Shrink model to first 766 iterations.
CV Log Loss Score: 0.777231

----------------- Fold 1 --------

In [77]:
# from sklearn.metrics import plot_confusion_matrix
# plt.rcParams['figure.figsize'] = (16,16)
# plot_confusion_matrix(cbt_clf, X_val, y_val)

In [48]:
sub = pd.read_csv('data/sample_submission.csv')

In [49]:
sub.iloc[:, 1:] = cat_pred_test
sub

,index,0,1,2
0,26457,0.224283,0.753219,0.022498
1,26458,0.113707,0.241356,0.644936
2,26459,0.140998,0.208679,0.650323
3,26460,0.204482,0.160781,0.634737
4,26461,0.074417,0.135251,0.790332
...,...,...,...,...
9995,36452,0.064593,0.134063,0.801344
9996,36453,0.175166,0.816542,0.008292
9997,36454,0.130185,0.245759,0.624055
9998,36455,0.150296,0.284317,0.565387


In [50]:
sub.to_csv('data/GH_1103_ver2_Cat.csv', index=False)

# 과적합되었음. 

## 잠깐 대기 (다음에 진행할 예정)

In [53]:
# 칼럼 리스트 분류
append_list = ['family_size', 'Nth_card', 'contact_num']
object_list = train.dtypes[train.dtypes == "object"].index.tolist()

for i in range(len(append_list)):
    object_list.append(append_list[i])

# OrdinalEncoder 시행
enc = OrdinalEncoder()

train[object_list] = enc.fit_transform(train[object_list], train['credit'])
test[object_list] = enc.transform(test[object_list])

In [54]:
numeric_list = ['income_total', 'begin_month', 'age_total', 'work_total', 'income_mean']

def scaler(df_1, df_2, col_name):
    scaler_ = MinMaxScaler()
    df_1[col_name] = scaler_.fit_transform(df_1[col_name])
    df_2[col_name] = scaler_.transform(df_2[col_name])
    
scaler(train, test, numeric_list)

In [55]:
names = ['train', 'test']

for name,dataset in zip(names, [train, test]):
    dataset.to_csv('{}final_set_{}.csv'.format(path,name))

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gender        26457 non-null  float64
 1   car           26457 non-null  float64
 2   reality       26457 non-null  float64
 3   income_total  26457 non-null  float64
 4   income_type   26457 non-null  float64
 5   edu_type      26457 non-null  float64
 6   family_type   26457 non-null  float64
 7   house_type    26457 non-null  float64
 8   occyp_type    26457 non-null  float64
 9   family_size   26457 non-null  float64
 10  begin_month   26457 non-null  float64
 11  credit        26457 non-null  float64
 12  age_total     26457 non-null  float64
 13  work_total    26457 non-null  float64
 14  ID            26457 non-null  float64
 15  Nth_card      26457 non-null  float64
 16  income_mean   26457 non-null  float64
 17  contact_num   26457 non-null  float64
dtypes: float64(18)
memory usag

In [58]:
train.drop(columns=['ID','occyp_type'], inplace=True)
test.drop(columns=['ID','occyp_type'], inplace=True)

In [62]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
a = ['income_type', 'edu_type', 'family_type', 'house_type', 'gender','car','reality']
enc.fit(train[a])
train[a] = enc.transform(train[a])
test[a] = enc.transform(test[a])


ValueError: Columns must be same length as key